In [1]:
%pwd

'e:\\LangChain project\\MedChatBot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'e:\\LangChain project\\MedChatBot'

In [4]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [5]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [6]:
embeddings = download_hugging_face_embeddings()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13236\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
e:\LangChain project\MedChatBot\medchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#test the embeddings
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medchatbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

e:\LangChain project\MedChatBot\medchatbot\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at db_data_2025-04.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


{
    "name": "medchatbot",
    "metric": "cosine",
    "host": "medchatbot-1uz26v7.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [10]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [11]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

NameError: name 'text_chunks' is not defined

In [11]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name='medchatbot',
    embedding=embeddings
)

In [12]:
docsearch

In [13]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [14]:
retrieved_docs = retriever.invoke("What is Acne?")

In [15]:
retrieved_docs

[Document(id='5d91da0a-50f4-43d0-8aef-44e51c345e1f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='f9d0f1c8-799c-430b-8308-91f3a5e3a525', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nat

In [16]:
GEMINI_API_KEY=os.environ.get('GEMINI_API_KEY')
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    google_api_key=GEMINI_API_KEY,
    temperature=0.2,
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    '''
    You are a medical question-answering assistant.  
    Use the retrieved context as your primary source of truth.  
    - If the context does not provide enough information, say "I don't know" or supplement with medically sound general knowledge.  
    - Provide clear, medically accurate explanations. Responses may be short or long depending on the complexity of the question.  
    - Vary your wording so repeated questions do not produce identical answers.  
    - When the question involves diagnosis, treatment, or personal medical decisions, remind the user to consult a qualified healthcare professional.  
    '''
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [20]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response['answer'])

{'input': 'what is Acromegaly and gigantism?', 'context': [Document(id='7ac47c74-e957-4412-ab91-ba8ebe483b9c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 45.0, 'page_label': '46', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='Whitehouse Station, NJ: Merck Research Laboratories,\n1997.\nLarsen, D. E., ed. Mayo Clinic Family Health Book.New York:\nWilliam Morrow and Co., Inc., 1996.\nJohn T. Lohr, PhD\nAcromegaly and gigantism\nDefinition\nAcromegaly is a disorder in which the abnormal\nrelease of a particular chemical from the pituitary gland\nin the brain causes increased growth in bone and soft tis-\nsue, as well as a variety of other disturbances throughout\nthe body. This chemical released from the pituitary gland'), Document(id='62b80ca8-a8ef-4fce-829d-a6a6c8ee79cd', metadata={'creationdate': '2004-12-18T17:

In [23]:
response = rag_chain.invoke({"input": "what's acne?"})
print("Response type:", type(response))
print("Response content:", response)
if isinstance(response, dict) and "answer" in response:
    print("Answer:", response["answer"])
else:
    print("Không tìm thấy key 'answer' trong response.")

Response type: <class 'dict'>
Response content: {'input': "what's acne?", 'context': [Document(id='5d91da0a-50f4-43d0-8aef-44e51c345e1f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(id='f9d0f1c8-799c-430b-8308-91f3a5e3a525', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder